In [ ]:
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

In [ ]:
sys.path.insert(0,"/home/container/notebooks/missions-cways/mobivia-revenue-forecast/mobivia_revenue_forecast/src")

#from random_draw_selection_feature import random_draw_selection_feature
from get_score_params_predict import get_score_params_predict
from evaluate_prediction import evaluate_prediction
from evaluate_prediction import evaluate_one_leave_out
from get_clean_feature_and_ca import get_clean_feature_and_ca
from display_model import display_model
from display_curve import display_curve
from exhaustive_selection_feature import exhaustive_selection_feature

# Constantes

In [ ]:
predict_year_list=[2021, 2022]
train_year_list=[2000, 2001,2002,2003,2004,2005,2006,2007,2008,2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,2017,2018, 2019, 2020]
to_keep_index_list=[k for k in range(2000, 2023)]#pour l'étape de pré-traitement : toutes les lignes dont on aura besoin
variable_number=3
#draw_nb=10
nb_model_to_select=100
growth_rate_bool=True #True pour avoir le modèle en taux de croissance
ca_col_str="NOR_CA_TOT_ACTU"

GLOBAL_MARKET_VAR_LIST=[
    "REV_DISPO_BRUT",
    "PA_REV_DISPO_BRUT",
    "PA_MENAGE", 
    "IND_PRIX_CONSO",
    "CONSO_FINALE_MENAGES",
    "PIB",
    "CHOMAGE",
    "TX_EPARGNE",
    "CONF_MENAGES",
    "ICA",
    "CA_E_COMMER_INIT"
]
"""AUTO_MARKET_VAR_LIST=[
    "NORAUTO_INVEST_PUB",
    "VOL_PARC",
    "VP03",
    "VP45",
    "VP67",
    "VP810",
    "VP1115",
    "VP610",
    "TXVP03",
    "TXVP45",
    "TXVP67",
    "TXVP810",
    "TXVP1115",
    "TXVP610",
    "IMMAT_VN",
    "IMMAT_VO"
]"""
AUTO_MARKET_VAR_LIST=[
    "TXVP610",
    "VP610"
]
MOBILITY_VAR_LIST=[
    "MODAL_SHARE_VP",
    "MODAL_SHARE_RAIL"
    "VP_KM_ANNUEL",
    "CONSO_CARBURANT",
    "FREQ_CENTRE_COMMER",
    "IND_PRIX_UTILISATION_VEHICULE",
    "PRIX_GASOIL",
    "ECART_TEMPERATURE",
]

#enlever l'année 2000 de l'entrainement si growth rate bool est Vrai

# Import et pré-traitement

In [ ]:
file_path = "/home/container/notebooks/bases/missions-cways/mobivia/mobivia_revenue_forecast/inputs/Consolidation_data_all.xlsx"

In [ ]:
france_pdf = pd.read_excel(open(file_path, 'rb'),sheet_name='France',skiprows=5, index_col = 0)

In [ ]:
feature_pdf, ca_pdf=get_clean_feature_and_ca(france_pdf, ca_col_str, to_keep_index_list, growth_rate_bool)

In [ ]:
predict_year_list=[2021, 2022]
Y = ca_pdf.drop(predict_year_list, axis=0)
X = feature_pdf.drop(predict_year_list, axis=0)
print(X, Y)

In [ ]:
X=X.drop(['ECART_TEMPERATURE','CA_E_COMMER_INIT'], axis=1)

# lasso

In [ ]:
from sklearn import linear_model

clf = linear_model.Lasso(alpha=0.1)
clf.fit(X, Y)
print(clf)
#Lasso(alpha=0.1)
print(clf.coef_)
print(clf.intercept_)

In [ ]:
my_alphas = np.array([0.0001,0.0002,0.0003,0.0005,0.00075,0.001,0.00125,0.0015,0.00175,0.002,0.0025,0.005,0.01])
#my_alphas = np.array([0.001,0.01,0.02,0.025,0.05,0.1,0.25,0.5,0.8,1.0])

from sklearn.linear_model import lasso_path
alpha_for_path, coefs_lasso, _ = lasso_path(X,Y,alphas=my_alphas)

In [ ]:
#jeu de couleurs pour faire joli
import matplotlib.cm as cm
couleurs = cm.rainbow(np.linspace(0,1,16))
#graphique lasso path (une courbe par variable)
for i in range(coefs_lasso.shape[0]):
    plt.plot(alpha_for_path,coefs_lasso[i,:])#,c=couleurs[i]

plt.xlabel('Alpha')
plt.ylabel('Coefficients')
plt.title('Lasso path')
plt.show()

In [ ]:
nbNonZero =np.apply_along_axis(func1d=np.count_nonzero,arr=coefs_lasso,axis=0)

In [ ]:
print(pd.DataFrame({'alpha':alpha_for_path,'Nb non-zero coefs':nbNonZero}))

In [ ]:
plt.plot(alpha_for_path,nbNonZero)
plt.xlabel('Alpha')
plt.ylabel('Nb. de variables')
plt.title('Nb. variables vs. Alpha')
plt.show()

In [ ]:
nom_var = X.columns
#coefficients pour alpha=0.25 (colonne n°3)
coefs25 = coefs_lasso[:,9]
#affichage des coefficients pour alpha = 0.25
print(pd.DataFrame({'Variables':nom_var,'Coefficients':coefs25}))

In [ ]:
chosen_cols=["CONSO_FINALE_MENAGES", "CHOMAGE", "PRIX_GASOIL", "TXVP810"]

# Prédiction, score, paramètres

In [ ]:
param_dict, score, estimated_serie = get_score_params_predict(feature_pdf[chosen_cols], ca_pdf, [2021, 2022], intercept_bool=True)

In [ ]:
print("score", score)
print("params_dict", param_dict)

In [ ]:
compare_results_df = pd.DataFrame(
    list(zip(ca_pdf.tolist(), estimated_serie)),
    columns = ['real_values', 'estimated_values'],
    index = ca_pdf.index
)
print(compare_results_df)

In [ ]:
plt.figure(figsize=(12,8))
compare_results_df.drop([2021, 2022], axis=0).plot()
compare_results_df.plot()
plt.xlabel("Année")
plt.ylabel("Chiffre d'affaires")
plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot

"""# Load a regression dataset
X, y = load_concrete()

# Create the train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)"""

# Instantiate the linear model and visualizer
model = LinearRegression()
visualizer = ResidualsPlot(model)
X_train=X[chosen_cols]
y_train=Y
visualizer.fit(X_train, y_train)  # Fit the training data to the visualizer
#visualizer.score(X_test, y_test)  # Evaluate the model on the test data
visualizer.show()                 # Finalize and render the figure

In [ ]:
visualizer = ResidualsPlot(model, hist=False, qqplot=True)
visualizer.fit(X_train, y_train)
#visualizer.score(X_test, y_test)
visualizer.show()

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import ProbPlot
plt.style.use('seaborn') # pretty matplotlib plots
plt.rc('font', size=14)
plt.rc('figure', titlesize=18)
plt.rc('axes', labelsize=15)
plt.rc('axes', titlesize=18)

In [ ]:
from sklearn.datasets import load_boston

boston = load_boston()

X = X_train
y = y_train

# generate OLS model
model = sm.OLS(y, sm.add_constant(X))
model_fit = model.fit()


# create dataframe from X, y for easier plot handling
dataframe = pd.concat([X, y], axis=1)

In [ ]:
# model values
model_fitted_y = model_fit.fittedvalues
# model residuals
model_residuals = model_fit.resid
# normalized residuals
model_norm_residuals = model_fit.get_influence().resid_studentized_internal
# absolute squared normalized residuals
model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
# absolute residuals
model_abs_resid = np.abs(model_residuals)
# leverage, from statsmodels internals
model_leverage = model_fit.get_influence().hat_matrix_diag
# cook's distance, from statsmodels internals
model_cooks = model_fit.get_influence().cooks_distance[0]

plot_lm_1 = plt.figure()
plot_lm_1.axes[0] = sns.residplot(model_fitted_y, dataframe.columns[-1], data=dataframe,
                          lowess=True,
                          scatter_kws={'alpha': 0.5},
                          line_kws={'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals');

In [ ]:
#kolmogorov
#stats.kstest(feature_pdf, "norm")
#levene
#stats.levene(feature_pdf[col].to_numpy())
#shapiro
#feature_pdf.apply(stats.shapiro)

In [ ]:
from scipy import stats

stats.shapiro(model_residuals)

In [ ]:
model_residuals.to_numpy()

In [ ]:
model_fitted_y.to_numpy()
model_fitted_y.shape

In [ ]:
import statsmodels.api as sm
test = sm.stats.het_breuschpagan(model_residuals.to_numpy(), X.to_numpy())

In [ ]:
test

In [ ]:
X